In [4]:
import sys
import os
from crewai import Agent, Crew , Process, Task, LLM 
from typing import List, Literal 
from pydantic import BaseModel, Field 
from crewai.tools.structured_tool import CrewStructuredTool
import requests
from typing import Dict, Any

Sample Crew Structured Tools

In [ ]:
# Define the schema for the tool's input using Pydantic
class APICallInput(BaseModel):
    endpoint: str
    parameters: dict

# Wrapper function to execute the API call
def tool_wrapper(*args, **kwargs):
    # Here, you would typically call the API using the parameters
    # For demonstration, we'll return a placeholder string
    return f"Call the API at {kwargs['endpoint']} with parameters {kwargs['parameters']}"

# Create and return the structured tool
def create_structured_tool():
    return CrewStructuredTool.from_function(
        name='Wrapper API',
        description="A tool to wrap API calls with structured input.",
        args_schema=APICallInput,
        func=tool_wrapper,
    )

# Example usage
structured_tool = create_structured_tool()

# Execute the tool with structured input
result = structured_tool._run(**{
    "endpoint": "https://example.com/api",
    "parameters": {"key1": "value1", "key2": "value2"}
})
print(result)  # Output: Call the API at https://example.com/api with parameters {'key1': 'value1', 'key2': 'value2'}

Second with Full API call 

In [ ]:


# Step 1: Define the schema for the tool's input using Pydantic
class APICallInput(BaseModel):
    endpoint: str
    parameters: Dict[str, Any]

# Step 2: Wrapper function to execute the API call and return a dictionary
def tool_wrapper(*args, **kwargs) -> Dict[str, Any]:
    endpoint = kwargs['endpoint']
    parameters = kwargs['parameters']
    
    try:
        # Make the API call (using JSONPlaceholder for demo)
        response = requests.get(endpoint, params=parameters, timeout=5)
        
        if response.status_code == 200:
            return {
                "status": "success",
                "data": response.json(),
                "endpoint_called": endpoint,
                "parameters_used": parameters
            }
        else:
            return {
                "status": "error",
                "message": f"API call failed with status {response.status_code}",
                "endpoint_called": endpoint,
                "parameters_used": parameters
            }
    except Exception as e:
        return {
            "status": "error",
            "message": str(e),
            "endpoint_called": endpoint,
            "parameters_used": parameters
        }

# Step 3: Create the structured tool
def create_structured_tool():
    return CrewStructuredTool.from_function(
        name='WrapperAPI',
        description="A tool to make API calls with structured input and return a dictionary response.",
        args_schema=APICallInput,
        func=tool_wrapper,
    )

# Step 4: Example usage
def main():
    # Create the structured tool
    structured_tool = create_structured_tool()

    # Define sample input
    input_data = {
        "endpoint": "https://jsonplaceholder.typicode.com/posts",
        "parameters": {"userId": 1}
    }

    # Execute the tool
    result = structured_tool._run(**input_data)

    # Print the result
    print("API Call Result:")
    print(f"Status: {result['status']}")
    print(f"Endpoint: {result['endpoint_called']}")
    print(f"Parameters: {result['parameters_used']}")
    if result['status'] == "success":
        print(f"Data (first 2 items): {result['data'][:2]}")
    else:
        print(f"Error: {result['message']}")

if __name__ == "__main__":
    main()